In [ ]:
#!python3 -m pip install tensorflow-macos

In [27]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [14]:
df = pd.read_csv('data/FinalBalancedDataset.csv')

,Unnamed: 0,Toxicity,tweet
0,0,0,@user when a father is dysfunctional and is s...
1,1,0,@user @user thanks for #lyft credit i can't us...
2,2,0,bihday your majesty
3,3,0,#model i love u take with u all the time in ...
4,4,0,factsguide: society now #motivation


# Vectorisation

Nous allons maintenant vectorisé les tweets, cela signifie donner une valeur numerique à chaques mot.
Ces mots serons alors dans un dictionnaire dont on va décider de la capacité de celui (ici je vais prendre 150 000).
On representera les tweets sous forme de tableau numpy. 
> Le dictionnaire que nous faisons est un vocabulaire compris uniquement par le model, on appel cela la tokenisation

In [26]:
vecteur = TextVectorization(max_tokens=150_000,output_sequence_length=1300,output_mode='int')
# le vecteur apprend mtn notre vocabulaire 
vecteur.adapt(df['tweet'].values)
# Il faut mtn vectorisé nos tweets
text_vectorise = vecteur(df['tweet'].values) 
text_vectorise

<tf.Tensor: shape=(56745, 1300), dtype=int64, numpy=
array([[    2,    34,     4, ...,     0,     0,     0],
       [    2,     2,   233, ...,     0,     0,     0],
       [  115,    27,  4880, ...,     0,     0,     0],
       ...,
       [  389,  5936,   225, ...,     0,     0,     0],
       [ 9865,    56,  1185, ...,     0,     0,     0],
       [18062, 44065, 63266, ...,     0,     0,     0]])>

# Creation du dataset
Maintenant que le vocabulaire existe et que les tweets sont vectoriser, il nous reste plus qu'a creer le dataset afin de l'envoyer au model

In [36]:
# On cree le dataset avec les tweets vectorisés et les labels (on ajoute des options supplémentaire qui vont servire a l'entrainement)
dataset = tf.data.Dataset.from_tensor_slices((text_vectorise, df['Toxicity'].values)).cache().shuffle(60000).batch(10).prefetch(8)
# Ici on va reperatir le contenu de notre dataset avec la partie entrainement 80%, evaluation 10% et test 10%
octante = int(len(dataset)*.8)
dix = int(len(dataset)*.1)
train,val,test = (dataset.take(octante),dataset.skip(octante).take(dix),dataset.skip(octante+dix).take(dix))

# Model 
le dataset est maintenant configuré, il est temps de creer notre model, de l'entrainer et de l'evaluer.
Dans le cadre du NLP et dans notre cas la compréhension de phrase de leur sens, il va falloir configurer le model de manière spécifique: 
- le nombre de couche egal au nombre max mot dans notre dictionnaire (embedding)
- une bidirectionnalité très importatn pour le sens des phrases
- Des couches intermédiaies qui connectent l'ensemble des couches   
- Enfin la couche final avec la fonction sigmoid qui va nous renvoyer une valeur entre 0 et 1

In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense

model = Sequential()
model.add(Embedding(150_001, 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [50]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy())
res_train = model.fit(train, epochs=1, validation_data=val)

4540/4540 [==============================] - 1137s 250ms/step - loss: 0.1992 - val_loss: 0.0966
